In [1]:
"""
add tags to a corpusfile (output of clean_text.py)

"""

import pandas as pd
import os
from simplediff import diff
from pythainlp.tokenize import word_tokenize

# get data from biased dataset 
# format : [text1, text2]
# text1 คือ ข้อมูลยังไม่เป็นกลาง
# text2 คือ ข้อมูลที่เป็นกลางแล้ว
schema_df = pd.read_csv('../dataset/LAW/biased.csv' ,encoding="utf-8-sig")

# หาผลต่างระหว่างการแก้ไข โดยใช้ Simplediff 
# output : [operater,text token]
def word_token(schema_df):
    token_diff = []
    data1 = schema_df.text1
    data2 = schema_df.text2
    for text1, text2 in zip(data1, data2):
        text_token1 = word_tokenize(text1)
        text_token2 = word_tokenize(text2)
        token_diff.append(diff(text_token1, text_token2))
        print(token_diff)
    return token_diff
    
# ใช้ tag e และ d จากเครื่องหมายที่ได้จาก simplediff 
# e = ['-','+']
# d = ['-']
def tag_data(tokdiff_data):
    out = []
    prev_tag = ['*']
    prev_words = []
    tag = []
    i = 0
    d = 0
    e = 0
    for tag, chunk in tokdiff_data:
        i += 1
        if tag == '+' and prev_tag[-1] == '-':
            word = '[e]' + ''.join(prev_words[-1]).strip() + '[/e]'
            if word != '[e][/e]':
                out += [word]
                e += 1
        elif tag == '=':
            if prev_tag[-1] == '-':
                word = '[d]' + ''.join(prev_words[-1]).strip() + '[/d]'
                if word != '[d][/d]':
                    out += [word]
                    d += 1
                out += chunk
            else :
                out += chunk
        elif i == len(tokdiff_data) and tag == '-':
            word = '[d]' + ''.join(chunk).strip() + '[/d]'
            if word != '[d][/d]':
                out += [word]
                d += 1
        prev_tag.append(tag)
        prev_words.append(chunk)
    return out

In [6]:
token_diff =  word_token(schema_df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
# convert array to string and tag bias
get_join = []
for i in range(len(token_diff)):
    out =  tag_data(token_diff[i])
#     print(''.join(out))
    get_join.append(''.join(out))
    get_join.append('')

In [12]:
for i in range(20) :
    print(get_join[i])

ในบันทึก[d]ย่อ[/d]ที่[e]น่าเศร้า[/e]เมื่อวันที่ 15 กันยายน 2547 จอห์นนี่[e]ราโมน[/e] ([e]ซึ่งแสดง[/e]ในตอนนี้กับราโมนส์) เสียชีวิต

เมื่อไม่นานมานี้ Novalja มีชื่อเสียงเพราะชายหาด Zre [d]ที่สวยงาม[/d]ซึ่งเป็นหนึ่งในเขตปาร์ตี้ฤดูร้อนที่ใหญ่ที่สุดในยุโรป

ไอดอลปรากฏตัวในนักเล่าเรื่อง VH1 และออกซีดียอดฮิตที่ยิ่งใหญ่ที่สุด[d]ที่ประสบความสำเร็จอย่างมาก[/d]ในปี 2544

เขาเปลี่ยนหยินชั่วร้ายเพราะเธอจะเดทกับเขา[d]ชั่วร้าย[/d]เมื่อเธอชั่วร้าย

Menino ได้พบคำวิจารณ์จากกลุ่มเสรีภาพพลเมืองเกี่ยวกับประเด็นต่าง ๆ ของการพูดฟรีรวมถึงการปรากฏตัวของตำรวจที่หนักและ[e]กดขี่[/e]ในช่วงการประชุมประชาธิปไตยแห่งชาติปี 2547

ตั้งแต่เดือนมกราคม 2009 เป็นต้นไปขณะที่ Abbondanzieri กลับไปที่ Boca Juniors Jacobo ได้รับการคืนสถานะเป็นผู้เริ่มต้นปรากฏตัวครั้งแรกในวันที่ 25 ใน[e]บ้าน[/e] 5-1 [e]ที่ประสบความสำเร็จ[/e]กับ Sporting de Gijn

ชื่อของผู้ปกครองมาซิโดเนียตอนบนมักจะไม่[e]ใช่[/e]กรีก (Arhabaios, Sirrhas) และนักวิชาการเช่น Eugene Borza ได้ใช้สิ่งนี้เพื่อยืนยันว่าชาวมาซิโดเนีย[d]ของมาซิโดเนีย[/d]ตอนบนยังคงมีชื่อมาซ

In [14]:
# save file
with open('../dataset/tagged_data/bias_data/tagged_data.txt',"w",  encoding="utf-8-sig") as f:
    i=0
    while i<len(get_join):
        for j in get_join[i]:
            f.write(j)
        if i+1<len(get_join):
            f.write("\n")
        i+=1

# TEST

In [3]:
text1 = '''ในเดือนพฤศจิกายน 2555 ฮาร์ตได้รับรอง mitt romney ให้เป็นประธานาธิบดี อย่างน่าเสียดาย'''
text2 = '''ฝรั่งเศสเริ่มขยายตัวในแอฟริกาตะวันตกในช่วงปลายทศวรรษ 1870 โดยผลักไปทางตะวันออกจากเซเนกัลเพื่อพยายามไปถึงต้นน้ำลำธารของแม่น้ำไนล์ซึ่งปัจจุบันเป็นประเทศซูดาน'''
text_token1 = word_tokenize(text1)
text_token2 = word_tokenize(text2)
tok_diff = diff(text_token1,text_token2)
print(text_token1)
print("------------")
print(text_token2)
print("------------")
print(tok_diff)

['ใน', 'เดือน', 'พฤศจิกายน', ' ', '2555', ' ', 'ฮา', 'ร์ต', 'ได้', 'รับรอง', ' ', 'mitt', ' ', 'romney', ' ', 'ให้', 'เป็น', 'ประธานาธิบดี', ' ', 'อย่าง', 'น่าเสียดาย']
------------
['ฝรั่งเศส', 'เริ่ม', 'ขยายตัว', 'ใน', 'แอฟริกา', 'ตะวัน', 'ตกใน', 'ช่วง', 'ปลาย', 'ทศวรรษ', ' ', '1870', ' ', 'โดย', 'ผลัก', 'ไป', 'ทาง', 'ตะวัน', 'ออกจาก', 'เซเนกัล', 'เพื่อ', 'พยายาม', 'ไป', 'ถึง', 'ต้นน้ำลำธาร', 'ของ', 'แม่น้ำ', 'ไนล์', 'ซึ่ง', 'ปัจจุบัน', 'เป็น', 'ประเทศ', 'ซูดาน']
------------
[('+', ['ฝรั่งเศส', 'เริ่ม', 'ขยายตัว']), ('=', ['ใน']), ('-', ['เดือน', 'พฤศจิกายน']), ('+', ['แอฟริกา', 'ตะวัน', 'ตกใน', 'ช่วง', 'ปลาย', 'ทศวรรษ']), ('=', [' ']), ('-', ['2555']), ('+', ['1870']), ('=', [' ']), ('-', ['ฮา', 'ร์ต', 'ได้', 'รับรอง', ' ', 'mitt', ' ', 'romney', ' ', 'ให้']), ('+', ['โดย', 'ผลัก', 'ไป', 'ทาง', 'ตะวัน', 'ออกจาก', 'เซเนกัล', 'เพื่อ', 'พยายาม', 'ไป', 'ถึง', 'ต้นน้ำลำธาร', 'ของ', 'แม่น้ำ', 'ไนล์', 'ซึ่ง', 'ปัจจุบัน']), ('=', ['เป็น']), ('-', ['ประธานาธิบดี', ' ', 'อย่าง', 'น่าเสียดาย'

In [4]:
text1

'ในเดือนพฤศจิกายน 2555 ฮาร์ตได้รับรอง mitt romney ให้เป็นประธานาธิบดี อย่างน่าเสียดาย'

In [5]:
text2

'ฝรั่งเศสเริ่มขยายตัวในแอฟริกาตะวันตกในช่วงปลายทศวรรษ 1870 โดยผลักไปทางตะวันออกจากเซเนกัลเพื่อพยายามไปถึงต้นน้ำลำธารของแม่น้ำไนล์ซึ่งปัจจุบันเป็นประเทศซูดาน'